In [13]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
import cv2
import csv
import sys
import os
from PIL import Image
import numpy as np
import torch
from torchvision.transforms import Compose, ToTensor
from tqdm.notebook import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
import random
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import PIL.Image
from torchvision import datasets
from torchvision import transforms


In [14]:
# detect is gpu available.
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("cuda")
    device =torch.device('cuda:0')
else:
    print("cpu")
    device =torch.device("cpu")

cpu


In [15]:
data_transforms =   transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                    ])
    
class Image_data(Dataset):
    def __init__(self,label,data,data_transforms):
        self.data_path = data
        self.label_path = label
        self.data_transforms = data_transforms
    def __getitem__(self,index):

        trainX = np.load(self.data_path)
        trainY = np.load(self.label_path)
        image = PIL.Image.fromarray(np.uint8(trainX[index]))
        trainY = torch.Tensor(trainY)
        
        return data_transforms(image),trainY[index]

    def __len__(self):
        return len(np.load(self.label_path))

train_data = Image_data('./trainY.npy','./trainX.npy',data_transforms)


batch_size = 32
validation_split = .1
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(train_data, batch_size=batch_size,sampler=train_sampler)
val_loader = DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler)


In [16]:
test_transforms =   transforms.Compose([
                    transforms.Resize((224, 224)),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                ])
class Test_data(Dataset):
    def __init__(self,data,data_transforms):
        self.data_path = data
        self.data_transforms = data_transforms
    def __getitem__(self,index):

        trainX = np.load(self.data_path)
        color_img = cv2.cvtColor(trainX[index],cv2.COLOR_GRAY2RGB)
        image = PIL.Image.fromarray(np.uint8(color_img))

        
        return test_transforms(image)

    def __len__(self):
        return len(np.load(self.data_path))





test_data = Test_data('testX.npy',test_transforms)
test_loader = DataLoader(test_data , batch_size=batch_size)


In [17]:
class LeNetEncoder(nn.Module):
    """LeNet encoder model for ADDA."""

    def __init__(self):
        """Init LeNet encoder."""
        super(LeNetEncoder, self).__init__()

        self.restored = False

        self.encoder = nn.Sequential(

            nn.Conv2d(3, 32, kernel_size=5,stride = 2),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5,stride = 2),
            nn.Dropout2d(),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=5),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU(),
        )
        self.fc1 = nn.Linear(2048, 512)

    def forward(self, input):
        """Forward the LeNet."""
        conv_out = self.encoder(input)
        feat = self.fc1(conv_out.view(-1, 2048))
        return feat


class LeNetClassifier(nn.Module):
    """LeNet classifier model for ADDA."""

    def __init__(self):
        """Init LeNet encoder."""
        super(LeNetClassifier, self).__init__()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, feat):
        """Forward the LeNet classifier."""
        out = F.dropout(F.relu(feat), training=self.training)
        out = self.fc2(out)
        return out
    

class Discriminator(nn.Module):
    """Discriminator model for source domain."""

    def __init__(self, input_dims, hidden_dims, output_dims):
        """Init discriminator."""
        super(Discriminator, self).__init__()

        self.restored = False

        self.layer = nn.Sequential(
            nn.Linear(input_dims, hidden_dims),
            nn.ReLU(),
            nn.Linear(hidden_dims, hidden_dims),
            nn.ReLU(),
            nn.Linear(hidden_dims, output_dims),
            nn.LogSoftmax()
        )

    def forward(self, input):
        """Forward the discriminator."""
        out = self.layer(input)
        return out

In [ ]:
tgt_encoder = (torch.load("./feature_extractor.pt"))
src_encoder  = (torch.load("./feature_extractor.pt"))
critic = Discriminator(512,128,2)
"""Train encoder for target domain """
####################
# 1. setup network #
####################

# set train state for Dropout and BN layers
tgt_encoder.train()
critic.train()

# setup criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer_tgt = optim.Adam(tgt_encoder.parameters(),
                           lr = 1e-4,
                           betas=(0.5, 0.9))
optimizer_critic = optim.Adam(critic.parameters(),
                              lr = 1e-4,
                               betas=(0.5, 0.9))
len_data_loader = min(len(train_loader), len(test_loader))

####################
# 2. train network #
####################
epochs = 500
for epoch in range(epochs):
    # zip source and target data pair
    data_zip = enumerate(zip(train_loader, test_loader))
    for step, ((images_src, _), (images_tgt)) in data_zip:
        ###########################
        # 2.1 train discriminator #
        ###########################

        # make images variable
        images_src = (images_src).to(device)
        images_tgt = (images_tgt).to(device)

        # zero gradients for optimizer
        optimizer_critic.zero_grad()

        # extract and concat features
        feat_src = src_encoder(images_src)
        feat_tgt = tgt_encoder(images_tgt)
        feat_concat = torch.cat((feat_src, feat_tgt), 0)

        # predict on discriminator
        pred_concat = critic(feat_concat.detach())
        # prepare real and fake label
        label_src = torch.ones(feat_src.size(0)).long().to(device)
        label_tgt = torch.zeros(feat_tgt.size(0)).long().to(device)

        label_concat = torch.cat((label_src, label_tgt))

        
        # compute loss for critic
        print(pred_concat)
        loss_critic = criterion(pred_concat, label_concat)
        
        loss_critic.backward()

        # optimize critic
        optimizer_critic.step()

        pred_cls = torch.squeeze(pred_concat.max(1)[1])
        acc = (pred_cls == label_concat).float().mean()

        ############################
        # 2.2 train target encoder #
        ############################

        # zero gradients for optimizer
        optimizer_critic.zero_grad()
        optimizer_tgt.zero_grad()

        # extract and target features
        feat_tgt = tgt_encoder(images_tgt)

        # predict on discriminator
        pred_tgt = critic(feat_tgt)

        # prepare fake labels
        label_tgt = (torch.ones(feat_tgt.size(0)).long()).to(device)

        # compute loss for target encoder
        loss_tgt = criterion(pred_tgt, label_tgt)
        loss_tgt.backward()

        # optimize target encoder
        optimizer_tgt.step()

        #######################
        # 2.3 print step info #
        #######################
        print("Epoch [{}/{}] Step [{}/{}]:"
              "d_loss={:.5f} g_loss={:.5f} acc={:.5f}"
              .format(epoch + 1,
                      epochs,
                      step + 1,
                      len_data_loader,
                      loss_critic.data,
                      loss_tgt.data,
                      acc.data),end ='\r')

    #############################
    # 2.4 save model parameters #
    #############################


